In [2]:
#pip install rdflib
import rdflib
from rdflib import Namespace , Literal , URIRef
from rdflib.namespace import RDF , RDFS



In [3]:
# create an empty Graph
g = rdflib.Graph()

# parse a local RDF file by specifying the format
result = g.parse("25102022output.ttl")

# Distribution of cultural phenomena

In [ ]:
cq1 = '''
prefix data: <http://icondataset.org/> 
PREFIX icon: <https://w3id.org/icon/ontology/> 
prefix sim: <https://w3id.org/simulation/ontology/>
prefix dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
prefix crm: <http://www.cidoc-crm.org/cidoc-crm/>

SELECT DISTINCT ?meaning ?meaningLabel (count(distinct ?artwork) as ?tot) ?time WHERE {
  ?iconol icon:aboutWorkOfArt ?artwork;
  icon:recognizedIntrinsicMeaning ?intrinsic.
  ?intrinsic (icon:recognizedConceptualObject | icon:recognizedCulturalPhenomenon) ?meaning.
  ?meaning a icon:CulturalPhenomenon; rdfs:label ?meaningLabel .
  OPTIONAL {?cultObj crm:P65_shows_visual_item ?artwork.
          ?production crm:P108_has_produced ?cultObj;
          crm:P4_has_time_span ?time}
}
GROUP BY ?meaning 
ORDER BY DESC(?tot)
'''

cq1res = g.query(cq1)

# create a list with phenomena that appear in at least 5 artworks
well_represented_phenomena = []
well_represented_phenomena_and_counts = []
for row in cq1res:
  if int(row[2]) >= 9:
    well_represented_phenomena.append("<"+str(row[0])+">")
    well_represented_phenomena_and_counts.append((str(row[0]), row[2]))
    print(row[2], '-',row[1])

26 - Representation of classical content with contemporary formal motifs
23 - Renascences of classical antiquity in Western Art before the Renaissance
13 - Reintegration of classical style and content in the Italian Renaissance
12 - Iconographic evolution of classical subjects
12 - Iconographic evolution of the representation of the three-headed companion of Serapis
11 - Classical motifs recovery
10 - Classical subject matter represented in classical form
10 - The phenomenon called Principle of disjunction: whenever in the high and later Middle Ages a work of art borrows its form from a classical model, this form is almost invariably invested with a non-classical, normally Christian, significance; whenever in the high and later Middle Ages a work of art borrows its theme from classical poetry, legend, history or mythology, this theme is quite invariably presented in a non-classical, normally contemporary, form.
9 - Intrinsic classicism was created by the meeting of Gothic and Proto-Ren

Notes: they are a lot, which is ok. I'm wondering if some can be generalized, e.g. an artwork that exemplifies "Iconographic evolution of nudity" can also exemplify "Iconographic evolution of classical subjects". If so, can the artwork be associated to both phenomena?

In [6]:
# same query with general categories

cq1 = '''
prefix data: <http://icondataset.org/> 
PREFIX icon: <https://w3id.org/icon/ontology/> 
prefix sim: <https://w3id.org/simulation/ontology/>
prefix dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
prefix crm: <http://www.cidoc-crm.org/cidoc-crm/>

SELECT DISTINCT ?CFbroader ?CFbroaderLabel (count(distinct ?artwork) as ?tot) ?time WHERE {
  ?iconol icon:aboutWorkOfArt ?artwork;
  icon:recognizedIntrinsicMeaning ?intrinsic.
  ?intrinsic (icon:recognizedConceptualObject | icon:recognizedCulturalPhenomenon) ?meaning.
  ?meaning a icon:CulturalPhenomenon; crm:P2_has_type ?CFbroader. 
?CFbroader rdfs:label ?CFbroaderLabel.
  OPTIONAL {?cultObj crm:P65_shows_visual_item ?artwork.
          ?production crm:P108_has_produced ?cultObj;
          crm:P4_has_time_span ?time}
}
GROUP BY ?CFbroader 
ORDER BY DESC(?tot)
'''

cq1res = g.query(cq1)

# create a list with phenomena that appear in at least 5 artworks
well_represented_phenomena = []
well_represented_phenomena_and_counts = []
for row in cq1res:
  if int(row[2]) >= 9:
    well_represented_phenomena.append("<"+str(row[0])+">")
    well_represented_phenomena_and_counts.append((str(row[0]), row[2]))
    print(row[2], '-',row[1])

203 - Reception of Classical antiquity
115 - Iconographical evolution
87 - Formal or stylistic evolution
59 - Formal or stylistic phenomenon
51 - Iconographical interpretation
39 - Tendency
30 - Belief
24 - Formal or stylistic influence
17 - Cultural reinterpretation
15 - Artist's expression
15 - Iconographical influence
15 - Attitude
14 - Cultural influence
10 - Societal phenomenon


In [12]:
well_represented_phenomena

['<http://icondataset.org/reception-of-classical-antiquity>',
 '<http://icondataset.org/iconographical-evolution>',
 '<http://icondataset.org/formal-or-stylistic-evolution>',
 '<http://icondataset.org/formal-or-stylistic-phenomenon>',
 '<http://icondataset.org/iconographical-interpretation>',
 '<http://icondataset.org/tendency>',
 '<http://icondataset.org/belief>',
 '<http://icondataset.org/formal-or-stylistic-influence>',
 '<http://icondataset.org/cultural-reinterpretation>',
 '<http://icondataset.org/artists-expression>',
 '<http://icondataset.org/iconographical-influence>',
 '<http://icondataset.org/attitude>',
 '<http://icondataset.org/cultural-influence>',
 '<http://icondataset.org/societal-phenomenon>']

# Subjects of well represented phenomena

Level 2 subjects associated to the most well represented cultural phenomena

In [ ]:
cq2 = """
prefix d: <http://icondataset.org/> 
PREFIX icon: <https://w3id.org/icon/ontology/> 
prefix sim: <https://w3id.org/simulation/ontology/>
prefix dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
prefix crm: <http://www.cidoc-crm.org/cidoc-crm/>

SELECT DISTINCT ?meaning ?artwork ?subj2  WHERE {
  VALUES ?rel {icon:hasCharacter icon:hasEvent icon:hasNamedObject icon:hasPlace icon:hasPersonification icon:hasSymbol}
  VALUES ?meaning { """+ ' '.join(well_represented_phenomena) +"""}
  ?meaning ^icon:recognizedCulturalPhenomenon / ^icon:recognizedIntrinsicMeaning ?iconol .
  ?iconol icon:aboutWorkOfArt ?artwork .
  ?icrec icon:aboutWorkOfArt ?artwork; 
          a icon:IconographicalRecognition;
          icon:recognizedImage ?image.
  
  ?image ?rel ?subj2. 

  #OPTIONAL {
  #  ?icrec icon:aboutWorkOfArt ?artwork;
  #  a icon:IconographicalRecognition; icon:recognizedInvenzione ?subj2.
  #}

} 
""" 

cq2res = g.query(cq2)
#for row in cq2res:
#  print(row[0], '-',row[2], " -- meaning:", row[1])

In [ ]:
# same, takng cultural phenomena typologies


In [15]:
cq2 = """
prefix d: <http://icondataset.org/> 
PREFIX icon: <https://w3id.org/icon/ontology/> 
prefix sim: <https://w3id.org/simulation/ontology/>
prefix dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
prefix crm: <http://www.cidoc-crm.org/cidoc-crm/>

SELECT DISTINCT ?CFbroader ?CFbroaderLabel ?artwork ?subj2  WHERE {
  VALUES ?rel {icon:hasCharacter icon:hasEvent icon:hasNamedObject icon:hasPlace icon:hasPersonification icon:hasSymbol}
  VALUES ?CFbroader { """+ ' '.join(well_represented_phenomena) +"""}
  ?CFbroader ^crm:P2_has_type ?meaning.
  ?meaning ^icon:recognizedCulturalPhenomenon / ^icon:recognizedIntrinsicMeaning ?iconol .
  ?iconol icon:aboutWorkOfArt ?artwork .
  ?icrec icon:aboutWorkOfArt ?artwork; 
          a icon:IconographicalRecognition;
          icon:recognizedImage ?image.
 
  ?CFbroader rdfs:label ?CFbroaderLabel. 
  ?image ?rel ?subj2. 

  #OPTIONAL {
  #  ?icrec icon:aboutWorkOfArt ?artwork;
  #  a icon:IconographicalRecognition; icon:recognizedInvenzione ?subj2.
  #}

} 
""" 

cq2res = g.query(cq2)
for row in cq2res:
  print(row[0], '-',row[3], " -- meaning:", row[1])

http://icondataset.org/reception-of-classical-antiquity - http://icondataset.org/mars  -- meaning: Reception of Classical antiquity
http://icondataset.org/reception-of-classical-antiquity - http://icondataset.org/venus  -- meaning: Reception of Classical antiquity
http://icondataset.org/reception-of-classical-antiquity - http://icondataset.org/jupiter  -- meaning: Reception of Classical antiquity
http://icondataset.org/reception-of-classical-antiquity - http://icondataset.org/jupiter  -- meaning: Reception of Classical antiquity
http://icondataset.org/reception-of-classical-antiquity - http://icondataset.org/saint-peter  -- meaning: Reception of Classical antiquity
http://icondataset.org/reception-of-classical-antiquity - http://icondataset.org/people/virgil  -- meaning: Reception of Classical antiquity
http://icondataset.org/reception-of-classical-antiquity - http://icondataset.org/saturn  -- meaning: Reception of Classical antiquity
http://icondataset.org/reception-of-classical-antiq

http://icondataset.org/formal-or-stylistic-evolution - http://icondataset.org/ariadne  -- meaning: Formal or stylistic evolution
http://icondataset.org/formal-or-stylistic-evolution - http://icondataset.org/the-children-of-mercury  -- meaning: Formal or stylistic evolution
http://icondataset.org/formal-or-stylistic-evolution - http://icondataset.org/david  -- meaning: Formal or stylistic evolution
http://icondataset.org/formal-or-stylistic-evolution - http://icondataset.org/pygmalion  -- meaning: Formal or stylistic evolution
http://icondataset.org/formal-or-stylistic-evolution - http://icondataset.org/argus  -- meaning: Formal or stylistic evolution
http://icondataset.org/formal-or-stylistic-evolution - http://icondataset.org/jupiter  -- meaning: Formal or stylistic evolution
http://icondataset.org/formal-or-stylistic-evolution - http://icondataset.org/apollo  -- meaning: Formal or stylistic evolution
http://icondataset.org/formal-or-stylistic-evolution - http://icondataset.org/thymbr

http://icondataset.org/tendency - http://icondataset.org/ixion  -- meaning: Tendency
http://icondataset.org/tendency - http://icondataset.org/bacchus  -- meaning: Tendency
http://icondataset.org/tendency - http://icondataset.org/centaur  -- meaning: Tendency
http://icondataset.org/tendency - http://icondataset.org/the-resurrected  -- meaning: Tendency
http://icondataset.org/tendency - http://icondataset.org/ariadne  -- meaning: Tendency
http://icondataset.org/tendency - http://icondataset.org/christ-as-a-child  -- meaning: Tendency
http://icondataset.org/tendency - http://icondataset.org/cupid  -- meaning: Tendency
http://icondataset.org/tendency - http://icondataset.org/apollo  -- meaning: Tendency
http://icondataset.org/tendency - http://icondataset.org/moses  -- meaning: Tendency
http://icondataset.org/belief - http://icondataset.org/bacchus  -- meaning: Belief
http://icondataset.org/belief - http://icondataset.org/mercury  -- meaning: Belief
http://icondataset.org/belief - http://i

http://icondataset.org/iconographical-influence - http://icondataset.org/mercury  -- meaning: Iconographical influence
http://icondataset.org/attitude - http://icondataset.org/thisbe  -- meaning: Attitude
http://icondataset.org/attitude - http://icondataset.org/europas-companions  -- meaning: Attitude
http://icondataset.org/attitude - http://icondataset.org/europa  -- meaning: Attitude
http://icondataset.org/attitude - http://icondataset.org/cupid  -- meaning: Attitude
http://icondataset.org/attitude - http://icondataset.org/apollo  -- meaning: Attitude
http://icondataset.org/attitude - http://icondataset.org/planetary-deities  -- meaning: Attitude
http://icondataset.org/attitude - http://icondataset.org/the-children-of-each-planet  -- meaning: Attitude
http://icondataset.org/attitude - http://icondataset.org/europas-companions  -- meaning: Attitude
http://icondataset.org/attitude - http://icondataset.org/roman-emperors  -- meaning: Attitude
http://icondataset.org/cultural-influence - 

http://icondataset.org/formal-or-stylistic-phenomenon - http://icondataset.org/apparition-of-christ  -- meaning: Formal or stylistic phenomenon
http://icondataset.org/formal-or-stylistic-phenomenon - http://icondataset.org/lamentation-of-christ  -- meaning: Formal or stylistic phenomenon
http://icondataset.org/formal-or-stylistic-phenomenon - http://icondataset.org/christ-resurrects-the-son-of-nain  -- meaning: Formal or stylistic phenomenon
http://icondataset.org/formal-or-stylistic-phenomenon - http://icondataset.org/the-abduction-of-europa  -- meaning: Formal or stylistic phenomenon
http://icondataset.org/formal-or-stylistic-phenomenon - http://icondataset.org/simeon-witnesses-the-presentation-of-christ  -- meaning: Formal or stylistic phenomenon
http://icondataset.org/formal-or-stylistic-phenomenon - http://icondataset.org/paris-rapes-helen  -- meaning: Formal or stylistic phenomenon
http://icondataset.org/iconographical-interpretation - http://icondataset.org/time-devouring-the-pa

...

http://icondataset.org/iconographical-interpretation - http://icondataset.org/torso-del-belvedere  -- meaning: Iconographical interpretation
http://icondataset.org/iconographical-interpretation - http://icondataset.org/love-stricken-soul  -- meaning: Iconographical interpretation
http://icondataset.org/iconographical-interpretation - http://icondataset.org/apollonian-swan  -- meaning: Iconographical interpretation
http://icondataset.org/iconographical-interpretation - http://icondataset.org/sella-gestatoria  -- meaning: Iconographical interpretation
http://icondataset.org/iconographical-interpretation - http://icondataset.org/caduceus  -- meaning: Iconographical interpretation
http://icondataset.org/iconographical-interpretation - http://icondataset.org/mercurys-winged-sandals  -- meaning: Iconographical interpretation
http://icondataset.org/iconographical-interpretation - http://icondataset.org/flame-of-divine-wisdom  -- meaning: Iconographical interpretation
http://icondataset.org/te

http://icondataset.org/iconographical-evolution - http://icondataset.org/fortune  -- meaning: Iconographical evolution
http://icondataset.org/iconographical-evolution - http://icondataset.org/chastity  -- meaning: Iconographical evolution
http://icondataset.org/iconographical-evolution - http://icondataset.org/tellus  -- meaning: Iconographical evolution
http://icondataset.org/iconographical-evolution - http://icondataset.org/truth  -- meaning: Iconographical evolution
http://icondataset.org/iconographical-evolution - http://icondataset.org/platonic-love  -- meaning: Iconographical evolution
http://icondataset.org/iconographical-evolution - http://icondataset.org/oceanus  -- meaning: Iconographical evolution
http://icondataset.org/iconographical-evolution - http://icondataset.org/eros-as-the-great-force-ruling-over-the-instincts-of-gods-man-and-animals  -- meaning: Iconographical evolution
http://icondataset.org/iconographical-evolution - http://icondataset.org/river-alpheus  -- meanin

http://icondataset.org/iconographical-influence - http://icondataset.org/sun  -- meaning: Iconographical influence
http://icondataset.org/iconographical-influence - http://icondataset.org/the-constellation-hercules  -- meaning: Iconographical influence
http://icondataset.org/iconographical-influence - http://icondataset.org/the-constellation-perseus  -- meaning: Iconographical influence
http://icondataset.org/iconographical-influence - http://icondataset.org/the-twins-constellation  -- meaning: Iconographical influence
http://icondataset.org/iconographical-influence - http://icondataset.org/the-constellation-hercules  -- meaning: Iconographical influence
http://icondataset.org/iconographical-influence - http://icondataset.org/blind-death  -- meaning: Iconographical influence
http://icondataset.org/iconographical-influence - http://icondataset.org/blind-synagogue  -- meaning: Iconographical influence
http://icondataset.org/attitude - http://icondataset.org/the-constellation-hercules  --

http://icondataset.org/iconographical-interpretation - http://icondataset.org/family-simple-happiness-of-primitive-civilization  -- meaning: Iconographical interpretation
http://icondataset.org/iconographical-interpretation - http://icondataset.org/venus-vulgaris-generative-force-that-creates-the-perishable-but-visible-and-tangible-images-of-beauty-of-earth  -- meaning: Iconographical interpretation
http://icondataset.org/iconographical-interpretation - http://icondataset.org/myrtle-everlasting-love  -- meaning: Iconographical interpretation
http://icondataset.org/iconographical-interpretation - http://icondataset.org/slave-human-soul-in-so-far-as-it-is-devoid-of-freedom  -- meaning: Iconographical interpretation
http://icondataset.org/iconographical-interpretation - http://icondataset.org/victory-human-soul-in-state-of-freedom  -- meaning: Iconographical interpretation
http://icondataset.org/iconographical-interpretation - http://icondataset.org/angel-smiling-while-carrying-the-coffin

http://icondataset.org/societal-phenomenon - http://icondataset.org/blue-hue-of-heaven  -- meaning: societal phenomenon
http://icondataset.org/societal-phenomenon - http://icondataset.org/weighing-with-balance-allusion-to-last-judgement  -- meaning: societal phenomenon
http://icondataset.org/societal-phenomenon - http://icondataset.org/mirror-introspection  -- meaning: Societal phenomenon
http://icondataset.org/societal-phenomenon - http://icondataset.org/jewelry-earthly-goods  -- meaning: societal phenomenon
http://icondataset.org/societal-phenomenon - http://icondataset.org/weighing-with-balance-weighing-of-souls  -- meaning: societal phenomenon
http://icondataset.org/societal-phenomenon - http://icondataset.org/mirror-introspection  -- meaning: societal phenomenon
http://icondataset.org/societal-phenomenon - http://icondataset.org/jewelry-earthly-goods  -- meaning: Societal phenomenon
http://icondataset.org/societal-phenomenon - http://icondataset.org/weighing-with-balance-weighing-

In [16]:
from collections import defaultdict , Counter

meanings_subjects = defaultdict(list)
# ?meaning ?artwork ?subj2
for row in cq2res:
  meanings_subjects[str(row['CFbroader'])].append(str(row['subj2']))

meanings_subjects_with_counts = {}
for k,v in meanings_subjects.items():
  for m,c in well_represented_phenomena_and_counts:
    if k == m:
      meanings_subjects_with_counts[k+" ("+c+")"] = dict(Counter(v).most_common())

# add count

meanings_subjects_with_counts

{'http://icondataset.org/reception-of-classical-antiquity (203)': {'http://icondataset.org/saturn': 16,
  'http://icondataset.org/cupid': 14,
  'http://icondataset.org/mercury': 11,
  'http://icondataset.org/venus': 8,
  'http://icondataset.org/jupiter': 8,
  'http://icondataset.org/apollo': 8,
  'http://icondataset.org/companion-of-serapis': 7,
  'http://icondataset.org/the-constellation-hercules': 6,
  'http://icondataset.org/blindfold-cupid': 5,
  'http://icondataset.org/saturn-devouring-a-living-child': 5,
  'http://icondataset.org/caduceus': 5,
  'http://icondataset.org/people/virgil': 4,
  'http://icondataset.org/satyr': 4,
  'http://icondataset.org/the-three-graces': 4,
  'http://icondataset.org/mary': 4,
  'http://icondataset.org/pluto': 4,
  'http://icondataset.org/bacchus': 4,
  'http://icondataset.org/era-sub-vulcano': 4,
  'http://icondataset.org/death': 4,
  'http://icondataset.org/mars': 3,
  'http://icondataset.org/silenus': 3,
  'http://icondataset.org/phaeton': 3,
  'h

In [23]:
frequent_subj_per_cf = {}
for item in meanings_subjects_with_counts.items():
    for pair in item[1].items(): 
        if int(pair[1]) >= 10:
            print(item[0], pair)
           # if item[0] not in frequent_subj_per_cf:
            #    frequent_subj_per_cf[item[0]] = {pair}
           # else: 
                
      

http://icondataset.org/reception-of-classical-antiquity (203) ('http://icondataset.org/saturn', 16)
http://icondataset.org/reception-of-classical-antiquity (203) ('http://icondataset.org/cupid', 14)
http://icondataset.org/reception-of-classical-antiquity (203) ('http://icondataset.org/mercury', 11)
http://icondataset.org/iconographical-evolution (115) ('http://icondataset.org/saturn', 16)
http://icondataset.org/iconographical-evolution (115) ('http://icondataset.org/cupid', 14)
http://icondataset.org/iconographical-interpretation (51) ('http://icondataset.org/cupid', 10)


In [ ]:
# same, with categories of subjects


cq2 = """
prefix d: <http://icondataset.org/> 
PREFIX icon: <https://w3id.org/icon/ontology/> 
prefix sim: <https://w3id.org/simulation/ontology/>
prefix dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
prefix crm: <http://www.cidoc-crm.org/cidoc-crm/>
prefix skos: <http://www.w3.org/2004/02/skos/core#>

SELECT DISTINCT ?CFbroader ?CFbroaderLabel ?artwork ?subj2 ?subj2Broader WHERE {
  VALUES ?rel {icon:hasCharacter icon:hasEvent icon:hasNamedObject icon:hasPlace icon:hasPersonification icon:hasSymbol}
  VALUES ?CFbroader { """+ ' '.join(well_represented_phenomena) +"""}
  ?CFbroader ^crm:P2_has_type ?meaning.
  ?meaning ^icon:recognizedCulturalPhenomenon / ^icon:recognizedIntrinsicMeaning ?iconol .
  ?iconol icon:aboutWorkOfArt ?artwork .
  ?icrec icon:aboutWorkOfArt ?artwork; 
          a icon:IconographicalRecognition. 
 {?icrec icon:recognizedImage ?image} UNION {?icrec icon:recognizedInvenzione ?invenzione}
 
  ?CFbroader rdfs:label ?CFbroaderLabel. 
  ?image ?rel ?subj2. 

  OPTIONAL {?subj2 skos:broader ?subj2Broader }
  OPTIONAL {?invenzione skos:broader ?invenzioneBroader }

} 
""" 

cq2res = g.query(cq2)
for row in cq2res:
  print(row[0], '-',row[3], "has broader subject: ", row[4], " -- meaning:", row[1])

Notes: there are too many subjects to see a pattern, I'd try to group them by type, manually, in a second iteration? e.g. greek gods, places, mythological figures, landscape, and so on.

Question: is it possible that you have many artworks without subjects of level 2? Look for instance at http://icondataset.org/CF1327, which is associated to 10 artworks, but all subjects appear in one artwork only. Answer: they appear in all the artworks only once. 

You can use the same method to query subjects of level 1.

In [ ]:
## First analysis: cf grouped per type, level 2 subj not grouped. 